In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.

# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
from os.path import isdir, join
import librosa
from tqdm import tqdm
# Any results you write to the current directory are saved as output.
import tensorflow as tf
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

['train', 'sample_submission.csv']
Default GPU Device: /device:GPU:0


In [3]:
# os.listdir("../input/dataasdfasd/")
# audio_file_path = "../input/dataasdfasd/output.wav"
# os.linesep()
# samps,sr = librosa.load(audio_file_path, mono=True, sr=None)
# mfcc = librosa.feature.mfcc(samps, sr = sr)z
# pad_width = max_pad - mfcc.shape[1]
# mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
audio_path = "../input/train/audio/"

In [4]:
# all audio dirs
dirs = [f for f in os.listdir(audio_path) if os.path.isdir(join(audio_path, f))]
dirs.sort()

In [5]:
print(dirs)

['_background_noise_', 'bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


In [6]:
import librosa.display

In [7]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 4))
# librosa.display.specshow(mfcc, x_axis='time')
# plt.colorbar()
# plt.title('MFCC')
# plt.tight_layout()

In [8]:
import os
import librosa
import numpy as np
class AudioFeatureDataset():
    def __init__(self,file_path):
        self.file_path = file_path
        self.labels = os.listdir(self.file_path)
        self.labels.remove('_background_noise_')
        self.target_labels = ['no', 'left', 'right', 'up', 'yes', 'down', 'stop', 'go', 'on', 'off','silence','unknown']
        self.data_dict = {}
        for tl in self.labels:
            files_dir = os.path.join(file_path,tl)
            all_audio_fp_s =[ os.path.join(files_dir,f) for f in os.listdir(files_dir)]
            self.data_dict[tl] = all_audio_fp_s
    
    def process(self,file,max_pad = 32):
        try:
            samps,sr = librosa.load(file, mono=True, sr=None)
            pad_len = 16000 - samps.shape[0]
            if pad_len  >= 0:
                samps= np.pad(samps,(0,pad_len),'constant')
            return np.array(samps[:16000])
        except:
            print(file)


    def get_dataset(self, include_background = False):
        labels =  []
        features = []
        for t in tqdm(self.labels):
            if (t == "_background_noise_"):
                pass
            if( t not in self.target_labels):
                for fp in self.data_dict[t]:
                    labels.append('unknown')
                    features.append(self.process(fp))
            else:
                 for fp in self.data_dict[t]:
                    labels.append(t)
                    features.append(self.process(fp))
        if include_background:
            _dir_path = (audio_path + "/_background_noise_")
            all_files_in_dir = os.listdir(_dir_path)
            all_files_in_dir.remove('README.md')
            all_sound_path = [ (audio_path + "/_background_noise_/" + x) for x in all_files_in_dir]
            all_sound_path
            all_samps = []
            for file in all_sound_path:
                samps,sr = librosa.load(file, mono=True, sr=None)
                k = int(samps.shape[0]/16000) * 16000
                samps =samps[:k].reshape(int(samps.shape[0]/16000),16000)
                for s in tqdm(range(len(samps))):
                    all_samps.append(samps[s])
                    labels.append('silence')
                    features.append(samps[s])
            all_samps = np.array(all_samps)
            
        return labels, features

In [9]:
a = AudioFeatureDataset(audio_path)

In [10]:
Y,x = a.get_dataset(True)

100%|██████████| 60/60 [00:00<00:00, 83220.32it/s]


In [11]:
import pandas as pd

In [12]:
x = np.array(x)
Y = np.array(Y)

In [13]:
del a

In [14]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced',np.unique(Y),Y)

In [15]:
class_weights

array([ 2.30037445,  2.287767  ,  2.30624026,  2.28487719,  2.3023264 ,
        2.29259963,  2.29259963, 13.63463149,  2.28007703,  0.13222991,
        2.28487719,  2.2829547 ])

In [16]:
# for i in range(len(x)):
#     pad_len = 16000 - x[i].shape[0]
#     x[i] = np.pad(x[i],(0,pad_len),'constant')

In [17]:
x = x.reshape(x.shape[0], 1, 16000) 

In [18]:
z  = [[1,2],[1,2]]
z = np.array(z)

In [19]:
x.shape

(65119, 1, 16000)

In [20]:
from sklearn.model_selection import train_test_split

def get_train_test(split_ratio=0.6, random_state=42):
    return train_test_split(x, Y, test_size= (1 - split_ratio), random_state=random_state, shuffle=True)

In [21]:
X_train, X_test, y_train, y_test = get_train_test()

In [22]:
del x

In [23]:
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 4))
# librosa.display.specshow(X_train[3], x_axis='time')
# plt.colorbar()
# plt.title('MFCC')
# plt.tight_layout()

In [24]:
from keras.utils.np_utils import to_categorical

Using TensorFlow backend.


In [25]:
del Y

In [26]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

In [27]:
labelencoder.fit_transform(y_test)

array([ 4,  9,  0, ...,  9,  9, 11])

In [28]:
mapping = dict(zip(labelencoder.classes_, range(len(labelencoder.classes_))))

In [29]:
mapping

{'down': 0,
 'go': 1,
 'left': 2,
 'no': 3,
 'off': 4,
 'on': 5,
 'right': 6,
 'silence': 7,
 'stop': 8,
 'unknown': 9,
 'up': 10,
 'yes': 11}

In [30]:
y_test = labelencoder.fit_transform(y_test)
mapping = dict(zip(labelencoder.classes_, range(len(labelencoder.classes_))))

In [31]:
mapping

{'down': 0,
 'go': 1,
 'left': 2,
 'no': 3,
 'off': 4,
 'on': 5,
 'right': 6,
 'silence': 7,
 'stop': 8,
 'unknown': 9,
 'up': 10,
 'yes': 11}

In [32]:
y_test

array([ 4,  9,  0, ...,  9,  9, 11])

In [33]:
y_train = labelencoder.fit_transform(y_train)
mapping = dict(zip(labelencoder.classes_, range(len(labelencoder.classes_))))

In [34]:
!pip install kapre

You are using pip version 19.0.3, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [35]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [44]:
import keras
import kapre
from keras.models import Sequential
from keras.layers import Dense,AveragePooling2D,BatchNormalization,SeparableConv2D
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
from kapre.augmentation import AdditiveNoise

# 6 channels (!), maybe 1-sec audio signal, for an example.
input_shape = (1,16000)
sr = 16000
model = Sequential()
# A mel-spectrogram layer
model.add(Melspectrogram(n_dft=512, n_hop=512, input_shape=input_shape,
                         padding='same', sr=sr, n_mels=128,
                         fmin=0.0, fmax=sr/2, power_melgram=1.0,
                         return_decibel_melgram=True,trainable_fb=False,
                         trainable_kernel=False,
                         name='trainable_stft'))
# Maybe some additive white noise.
model.add(AdditiveNoise(power=0.1))
# If you wanna normalise it per-frequency
model.add(Normalization2D(str_axis='freq')) # or 'channel', 'time', 'batch', 'data_sample'
# After this, it's just a usual keras workflow. For example..
# Add some layers, e.g., model.add(some convolution layers..)
# Compile the model
model.add(Conv2D(64, kernel_size=(20, 8), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2),dim_ordering="th"))
model.add(Dropout(0.25))
## Depth Seprable Pooling Layer - start
model.add(SeparableConv2D(64, kernel_size=(5, 5), activation='relu',dim_ordering="th"))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(1, 1), activation='relu',dim_ordering="th"))
model.add(BatchNormalization())
model.add(SeparableConv2D(64, kernel_size=(5, 5), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(1, 1), activation='relu'))
model.add(BatchNormalization())
## Depth Seprable pooling Layer - end
model.add(AveragePooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(12, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.adam(),metrics=['accuracy'])
model.summary() 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:28: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(pool_size=(2, 2), data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: Update your `SeparableConv2D` call to the Keras 2 API: `SeparableConv2D(64, kernel_size=(5, 5), activation="relu", data_format="channels_first")`
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:33: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, kernel_size=(1, 1), activation="relu", data_format="channels_first")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
trainable_stft (Melspectrogr (None, 128, 32, 1)        296064    
_________________________________________________________________
additive_noise_2 (AdditiveNo (None, 128, 32, 1)        0         
_________________________________________________________________
normalization2d_2 (Normaliza (None, 128, 32, 1)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 109, 25, 64)       10304     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 109, 12, 32)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 109, 12, 32)       0         
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 64, 8, 28)         9765      
__________

In [37]:
from keras.utils import to_categorical
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

In [38]:
# X_train.reshape(-1,13062,16000).shape

In [39]:

# X_test = X_test.reshape(X_test.shape[0], 20, 32, 1)

In [40]:
# a = X_train[0].reshape(1,16000)

In [41]:
# model.predict(np.array([a]))

In [42]:
from keras.optimizers import Optimizer
from keras import backend as K
import six
import copy
from six.moves import zip
from keras.utils.generic_utils import serialize_keras_object
from keras.utils.generic_utils import deserialize_keras_object
from keras.legacy import interfaces
class AdamW(Optimizer):
    """Adam optimizer.
    Default parameters follow those provided in the original paper.
    # Arguments
        lr: float >= 0. Learning rate.
        beta_1: float, 0 < beta < 1. Generally close to 1.
        beta_2: float, 0 < beta < 1. Generally close to 1.
        epsilon: float >= 0. Fuzz factor.
        decay: float >= 0. Learning rate decay over each update.
        weight_decay: float >= 0. Decoupled weight decay over each update.
    # References
        - [Adam - A Method for Stochastic Optimization](http://arxiv.org/abs/1412.6980v8)
        - [Optimization for Deep Learning Highlights in 2017](http://ruder.io/deep-learning-optimization-2017/index.html)
        - [Fixing Weight Decay Regularization in Adam](https://arxiv.org/abs/1711.05101)
    """

    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4,  # decoupled weight decay (1/6)
                 epsilon=1e-8, decay=0., **kwargs):
        super(AdamW, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.init_lr = lr # decoupled weight decay (2/6)
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            self.wd = K.variable(weight_decay, name='weight_decay') # decoupled weight decay (3/6)
        self.epsilon = epsilon
        self.initial_decay = decay

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        wd = self.wd # decoupled weight decay (4/6)

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))
        eta_t = lr / self.init_lr # decoupled weight decay (5/6)

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - eta_t * wd * p # decoupled weight decay (6/6)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'weight_decay': float(K.get_value(self.wd)),
                  'epsilon': self.epsilon}
        base_config = super(AdamW, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [45]:
model.fit(X_train, y_train, batch_size=100, epochs=50, verbose=1, validation_data=(X_test, y_test),shuffle=True)

Train on 39071 samples, validate on 26048 samples
Epoch 1/50
39071/39071 [==============================] - 14s 350us/step - loss: 0.9909 - acc: 0.7033 - val_loss: 0.6511 - val_acc: 0.7791
Epoch 2/50
39071/39071 [==============================] - 12s 313us/step - loss: 0.5565 - acc: 0.8177 - val_loss: 0.4704 - val_acc: 0.8425
Epoch 3/50
39071/39071 [==============================] - 12s 314us/step - loss: 0.4362 - acc: 0.8566 - val_loss: 0.3751 - val_acc: 0.8780
Epoch 4/50
39071/39071 [==============================] - 12s 314us/step - loss: 0.3569 - acc: 0.8822 - val_loss: 0.3378 - val_acc: 0.8910
Epoch 5/50
39071/39071 [==============================] - 12s 315us/step - loss: 0.3144 - acc: 0.8983 - val_loss: 0.3056 - val_acc: 0.9041
Epoch 6/50
39071/39071 [==============================] - 12s 319us/step - loss: 0.2867 - acc: 0.9049 - val_loss: 0.3020 - val_acc: 0.9050
Epoch 7/50
39071/39071 [==============================] - 12s 314us/step - loss: 0.2642 - acc: 0.9124 - val_loss: 0.

In [46]:
print(model.output.op.name)
print(model.input.op.name)

dense_2/Softmax
trainable_stft_input_1


In [47]:
model.output.op.name

'dense_2/Softmax'

In [48]:
model.save_weights('dscnn.h5',keras)

In [49]:
model.save("dscnn-model.h5")
import tensorflow as tf

In [ ]:
# from tensorflow.contrib import lite
# converter = lite.TFLiteConverter.from_keras_model_file( 'yo.h5')
# tfmodel = converter.convert()
# converter.allow_custom_ops=True
# open ("model.tflite" , "wb") .write(tfmodel)
